## Aim

Write a general ABC sampler, to understand the technique and to apply it to exoplanet occurrence problems.

In [2]:
import numpy as np

In [46]:
class ABCSampler:
    def __init__(self, prior, candidate_getter):
        '''
        prior : function
        Samples from the prior distribution over the parameters.
        Takes in nothing.
        Returns numpy.ndarray of parameters.

        candidate_getter : function
        Gets candidate samplers of the posterior distribution.
        Takes in parameters, as returned by prior.
        Returns function 'candidate' that samples from the candidate distribution, to be compared to true data, which
            Takes in the number of sample points to return.
            Returns numpy.ndarray of sample points of the desired shape.
        '''
        self.prior = prior
        self.candidate_getter = candidate_getter
        
    def sample(self, data, statistic=np.mean, max_iters=1000, threshold=1e-1):
        '''
        data : numpy.ndarray
        Data that we want to fit. 
        Arbitrary shape, but a row should match the return type of candidate.

        statistic : function
        The statistic on which to compare sampled data to true data.
        Takes in data array/matrix matching 'data' and the return type of candidate.
        Returns a scalar representing some measure of the data.

        iters : scalar
        The number of times to try and accept a candidate.

        threshold : scalar
        The maximum difference between the values of statistic(data) and statistic(candidate()) allowable to accept candidate.
        '''
        num_iters = 0
        while True:
            params = self.prior()
            candidate = self.candidate_getter(params)
            synthetic = candidate(data.shape[0])
            distance = abs(statistic(synthetic) - statistic(data))
            if distance <= threshold:
                print("Accepted parameters: ", params)
            num_iters += 1
            if num_iters > max_iters:
                return

In [47]:
# First simple example: a Gaussian fit you can do with MLE.
# Suppose we've sampled data from N(50, 10), but our best guess is it's N(N(40, 20), N(11, 2)).
prior = lambda: np.array([np.random.normal(40, 20), np.random.normal(11, 2)])
def candidate_getter(p):
    def candidate(size):
        return np.random.normal(*p, size)
    return candidate

gaussian_sampler = ABCSampler(prior, candidate_getter)
gaussian_sampler.sample(np.random.normal(50, 10, (100,)))

Accepted parameters:  [49.56738937 12.69001288]
Accepted parameters:  [49.84089257 12.33524337]
Accepted parameters:  [49.8904846  11.17047454]
Accepted parameters:  [50.16745596 11.28804709]
Accepted parameters:  [50.50100194  8.63290075]
Accepted parameters:  [48.97776997  8.05510468]
Accepted parameters:  [52.0818549  14.50223798]
Accepted parameters:  [49.05114373 12.33660459]


In [49]:
# Exp(22) but we estimate Exp(max(0, N(20, 2)))
prior = lambda: np.array([max(0, np.random.normal(20, 2))])
def candidate_getter(p):
    def candidate(size):
        return np.random.exponential(*p, size)
    return candidate

exp_sampler = ABCSampler(prior, candidate_getter)
exp_sampler.sample(np.random.exponential(22, (100,)), statistic=np.median, threshold=0.01)

Accepted parameters:  [21.76850258]


In [ ]:
# now, getting closer to exoplanet parameter fitting: a power-law

1. https://arxiv.org/pdf/1802.09720.pdf